In [2]:
import os
import requests
import re

To use the selenium and the ChromeDriver, you will have to:

- `conda install -c conda-forge selenium`

Then you need to download ChromeDriver, install it here: <br>
https://sites.google.com/a/chromium.org/chromedriver/downloads

Be sure you download the right Chrome version. <br>
If you don't know your chrome version, you just need to click at options, help, about chrome and then you will find the right version for you. 

Once you have the chromedriver downloaded, run thoses lines:

```
sudo mv chromedriver /usr/bin/chromedriver
sudo chown root:root /usr/bin/chromedriver
sudo chmod +x /usr/bin/chromedriver
```

In [5]:
from selenium import webdriver
options = webdriver.ChromeOptions()
options.add_argument('headless')
options.add_argument('window-size=1920x1080')
driver = webdriver.Chrome(chrome_options=options)
driver.get("https://www.wikiaves.com.br/midias.php?tm=s&t=s&s=11644&o=mp")

/home/alonkin/anaconda3/envs/development/lib/python3.7/site-packages/ipykernel_launcher.py:5: DeprecationWarning: use options instead of chrome_options
  """


In [6]:
title = driver.find_element_by_class_name('wa-registros-titulo')
complete_name = title.text.strip('Sons de ')
popular_name = complete_name.split(' ')[0]
cientific_name = complete_name[complete_name.find('(') + 1: complete_name.find(')')] 

print(complete_name)
print(popular_name)
print(cientific_name)

canário-da-terra (Sicalis flaveola)
canário-da-terra
Sicalis flaveola


In [16]:
audios = driver.find_elements_by_tag_name('audio')
for n, audio_src in enumerate(audios, 0):
    url = audio_src.get_attribute('src')
    r = requests.get(url, allow_redirects=True)
    filename = 'audios/' + popular_name + "_" + str(n) + ".mp3"
    open(filename, 'wb').write(r.content)